In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
import matplotlib as pl
import seaborn as sb
import math
import copy
import random
import re

In [2]:
data_np = pd.read_csv('data_new.csv')

In [3]:
data_np.head()

,avatar,bdate,blog_last,blog_total,friends_num,given_negative_marks,given_positive_marks,got_negative_marks,got_positive_marks,last_visit,...,visits_num,Интересы,Конфигурация компьютера,Местоположение,О себе,Реальное имя,Специализация,Стаж работы,Чем занимаетесь,last_visit_new
0,0.0,NaN,never,0.0,0.0,0.0,0.0,0.0,0.0,11.11.2017,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.11.2017
1,0.0,26 March 1998 (19),never,0.0,0.0,0.0,18.0,0.0,0.0,12.11.2017,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.11.2017
2,0.0,17 February 1996 (21),never,0.0,0.0,0.0,0.0,0.0,0.0,11.11.2017,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.11.2017
3,0.0,18 June 2001 (16),never,0.0,0.0,0.0,0.0,0.0,0.0,11.11.2017,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.11.2017
4,1.0,10 January 1992 (25),never,0.0,1.0,0.0,59.0,2.0,210.0,12.11.2017,...,214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.11.2017


In [4]:
data_np.columns

Index([u'avatar', u'bdate', u'blog_last', u'blog_total', u'friends_num',
       u'given_negative_marks', u'given_positive_marks', u'got_negative_marks',
       u'got_positive_marks', u'last_visit', u'msg_per_day', u'msg_total',
       u'name', u'rdate', u'referrals', u'reputation', u'url', u'visits_num',
       u'Интересы', u'Конфигурация компьютера', u'Местоположение', u'О себе',
       u'Реальное имя', u'Специализация', u'Стаж работы', u'Чем занимаетесь',
       u'last_visit_new'],
      dtype='object')

In [5]:
# Разбираемся с датой рождения - преобразуем дату рождения в возраст

In [6]:
def get_years(dt, missing='aver'):
    tst_arr = np.array(dt.bdate.values, dtype=str)
    left_br = np.char.find(tst_arr, '(')
    right_br = np.char.find(tst_arr, ')')
    right_br[right_br == -1] = 0
    for ind in xrange(tst_arr.size):
        tst_arr[ind] = tst_arr[ind][(left_br[ind] + 1):right_br[ind]]
    tst_arr[tst_arr == ''] = '0'
    tst_arr = tst_arr.astype(int)
    if(missing == 'aver'):
        tst_arr[tst_arr == 0] = np.average(tst_arr[tst_arr != 0])
    elif(missing == 'med'):
        tst_arr[tst_arr == 0] = np.median(tst_arr[tst_arr != 0])
    else:
        rnd_val = np.random.randint(0, 2, tst_arr.size)
        tst_arr[tst_arr == 0] = np.average(tst_arr[tst_arr != 0]) * rnd_val + \
                                np.median(tst_arr[tst_arr != 0]) * (1 - rnd_val)
    return tst_arr

In [7]:
data_np.bdate = get_years(data_np)

In [8]:
# Убираем NaN в аватаках

In [9]:
bad_avarar = data_np.avatar.isna() == True

In [10]:
data_np = data_np.loc[bad_avarar != True]

In [11]:
# Разбираемся с blog_last

In [12]:
bad_blog_last = (data_np.blog_last.isna() == True)

In [13]:
data_np = data_np.loc[bad_blog_last != True]

In [14]:
def get_blog_last(dt):
    tst_arr = np.array(dt.blog_last.values, dtype=str)
    tst_arr[tst_arr == 'never'] = '0'
    tst_arr[tst_arr != '0'] = '1'
    tst_arr = tst_arr.astype(int)
    return tst_arr

In [15]:
data_np.blog_last = get_blog_last(data_np)

In [16]:
# Blog_total - уже всё ок

In [17]:
# Friends_num - уже всё ок

In [18]:
# Marks... - уже всё ок

In [19]:
# Msg_per_day - уже всё ок

In [20]:
# Msg_total -> float

In [21]:
data_np.msg_total = data_np.msg_total.values.astype(float)

In [22]:
# name -> drop

In [23]:
data_np = data_np.drop('name', axis=1)

In [24]:
# rdate -> days after reg

In [25]:
def get_reg(dt):
    tst_arr = np.array(dt.rdate.values, dtype=str)
    rtime = np.empty(tst_arr.size, dtype=int)
    nt = datetime.strptime(datetime.now().strftime("%d.%m.%Y"), '%d.%m.%Y')
    for ind in xrange(tst_arr.size):
        rtime[ind] = (nt - datetime.strptime(dt.rdate.values[ind], '%d.%m.%Y')).days
    return rtime

In [26]:
data_np.rdate = get_reg(data_np)

In [27]:
# url -> drop

In [28]:
data_np = data_np.drop('url', axis=1)

In [29]:
# reputation -> float

In [30]:
data_np.reputation = data_np.reputation.values.astype(float)

In [31]:
# visits_num - 44 - некорректный профиль

In [32]:
data_np = data_np.loc[data_np.index != 44]

In [33]:
# Заменяем заполненость поля на 0/1

In [34]:
for ind in xrange(16, 24):
    index = data_np.iloc[:, ind].isna()
    tmp_arr = np.empty(index.size)
    tmp_arr[index == True] = 0
    tmp_arr[index != True] = 1
    data_np.iloc[:, ind] = tmp_arr

In [35]:
# Создаём массив целевой переменной

In [36]:
# 2489 - ещё один некорректный профиль

In [37]:
data_np = data_np.loc[data_np.index != 2489]

In [38]:
delta_bet_vis = np.empty(data_np.last_visit.size, dtype=int)
for ind in xrange(delta_bet_vis.size):
    lv = datetime.strptime(data_np.last_visit.values[ind], '%d.%m.%Y')
    lvn = datetime.strptime(data_np.last_visit_new.values[ind], '%d.%m.%Y')
    delta_bet_vis[ind] = (lvn - lv).days > 0

In [39]:
y_real = delta_bet_vis

In [40]:
data_np = data_np.drop('last_visit', axis=1)
data_np = data_np.drop('last_visit_new', axis=1)

In [42]:
X_plot = data_np.values
x = np.arange(X_plot.shape[1])
y = np.empty(X_plot.shape[1])
for i in xrange(X_plot.shape[1]):
    y[i] = np.sum(X_plot[:, i] != 0)
    print data_np.columns[i], y[i], np.sum((X_plot[:, i] != 0) * y_real == 1)

avatar 501.0 454
bdate 5248.0 3880
blog_last 106.0 100
blog_total 107.0 101
friends_num 353.0 332
given_negative_marks 82.0 80
given_positive_marks 2432.0 2113
got_negative_marks 84.0 82
got_positive_marks 1246.0 1123
msg_per_day 2961.0 2537
msg_total 3317.0 2803
rdate 5248.0 3880
referrals 139.0 128
reputation 1269.0 1145
visits_num 5248.0 3880
Интересы 295.0 268
Конфигурация компьютера 264.0 233
Местоположение 537.0 485
О себе 148.0 127
Реальное имя 623.0 551
Специализация 628.0 557
Стаж работы 381.0 343
Чем занимаетесь 376.0 337


In [707]:
# Нормализуем признаки

In [733]:
X_real = preprocessing.scale(data_np.values)
#pca = PCA(n_components=0.95)
#X_real = pca.fit_transform(X_real)

In [709]:
# Выделяем часть данных, которая не будет использоваться вплоть до окончания подборов параметров модели

In [710]:
X_real, X_real_saved, \
y_real, y_real_saved = train_test_split(X_real, y_real, test_size = 0.2)

In [711]:
# Отбирать некоррелированные признаки вручную - слишком сложно
# Примем за аксиому, что коррелированные признаки значительно понижают кажество предсказаний
# Будем случайно выбирать признаки и искать наилучшее сочетания

In [712]:
def try_get_best_feat(max_iter):
    roc_auc_best = 0
    rand_fea_best = []

    for i in xrange(max_iter):
        if i % 10 == 0:
            print i

        rand_fea = np.unique(np.random.choice(np.arange(X_real.shape[1]), random.randint(1, X_real.shape[1])))

        roc = 0
        X_real_tmp = X_real[:, rand_fea]
        y_real_tmp = y_real

        for j in range(5):
            X_real_tmp_train, X_real_tmp_test, \
            y_real_tmp_train, y_real_tmp_test = train_test_split(X_real_tmp, y_real_tmp, test_size = 0.3)

            model = RandomForestClassifier()
            model.fit(X_real_tmp_train, y_real_tmp_train)
            roc = roc + roc_auc_score(y_real_tmp_test, model.predict(X_real_tmp_test))
        roc = roc / 5

        if roc > roc_auc_best:
            roc_auc_best = roc_auc_score(y_real_tmp_test, model.predict(X_real_tmp_test))
            rand_fea_best = rand_fea

    return roc_auc_best, rand_fea_best 

In [713]:
#print try_get_best_feat(200)

In [714]:
# Результат - лучший результат показал набор - [6, 9, 11, 14, 15, 19] им и будем пользоваться

In [715]:
best_features = [6, 9, 11, 14, 15, 19]

In [716]:
X_real_best = X_real[:, best_features]
X_real_saved_best = X_real_saved[:, best_features]

In [717]:
for it in best_features:
    print it, data_np.columns[it]

6 given_positive_marks
9 msg_per_day
11 rdate
14 visits_num
15 Интересы
19 Реальное имя


In [718]:
# Попытаемся подобрать наиболее оптимальные глобальные параметры для модели
# Т.к. это, по сути, много параметрическая оптимизация, будем случайно выбирать параметры и искать оптимальную конфигурацию

In [719]:
def try_get_best(max_iter):
    roc_auc_best = 0
    rand_max_depth = 0
    rand_params_best = np.empty([6], dtype=int)

    for i in xrange(max_iter):
        if i % 10 == 0:
            print i
    
        roc = 0
        X_real_tmp = X_real_best
        y_real_tmp = y_real
    
    
        rand_params = np.empty([6], dtype=int)
        #max_feat
        rand_params[0] = random.randint(1, X_real_tmp.shape[1])
        #max_depth
        rand_params[1] = random.randint(1, 50)
        #min_samples_split
        rand_params[2] = random.randint(2, 50)
        #min_samples_leaf
        rand_params[3] = random.randint(1, 50)
        #max_leaf_nodes
        rand_params[4] = random.randint(X_real.shape[0] / 3, X_real.shape[0])
        #n_estimators
        rand_params[5] = random.randint(1, 20)

        for j in range(5):
            X_real_tmp_train, X_real_tmp_test, \
            y_real_tmp_train, y_real_tmp_test = train_test_split(X_real_tmp, y_real_tmp, test_size = 0.3)

            model = RandomForestClassifier(max_features=rand_params[0],
                                           max_depth = rand_params[1],
                                           min_samples_split = rand_params[2],
                                           min_samples_leaf=rand_params[3],
                                           max_leaf_nodes=rand_params[4],
                                           n_estimators=rand_params[5])
            model.fit(X_real_tmp_train, y_real_tmp_train)
            roc = roc + roc_auc_score(y_real_tmp_test, model.predict(X_real_tmp_test))
        roc = roc / 5
    
        if roc > roc_auc_best:
            roc_auc_best = roc_auc_score(y_real_tmp_test, model.predict(X_real_tmp_test))
            rand_params_best = rand_params

    return roc_auc_best, rand_params_best

In [720]:
#print try_get_best(20)

In [721]:
# Лучший результат -  max_features=3, max_depth = 7, min_samples_split = 3, min_samples_leaf = 12, n_estimators=12

In [722]:
# Разбиваем выборку на тестовую и тренировочную части

In [723]:
X_real_train, X_real_test, \
y_real_train, y_real_test = train_test_split(X_real_best, y_real, test_size = 0.3)

In [724]:
model = RandomForestClassifier(max_features=3, max_depth = 7, min_samples_split = 3, min_samples_leaf = 12, n_estimators=12)

In [725]:
model.fit(X_real_train, y_real_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=12, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [726]:
# Смотрим результат на отложенной выборке - довольно неплохо

In [727]:
print 'ROC_AUC:', roc_auc_score(y_real_saved, model.predict(X_real_saved_best))

ROC_AUC: 0.710142766616


In [728]:
print 'F-m:', f1_score(y_real_saved, model.predict(X_real_saved_best))

F-m: 0.9027695934


In [729]:
print np.sum(y_real_saved == 0)

278
